# Lab Control

In this tutorial, we take a variety of  different pieces of scientific equipment and use this 
[Jupyter](https://jupyter.org/) Notebook to control the devices using 
[Python](https://python.org/).

The devices which we are currently controlling in this tutorial are:

* [Keysight 33512B Waveform Generator](https://www.keysight.com/au/en/assets/7018-05928/data-sheets/5992-2572.pdf)
* [Rigol DS1074 Oscilloscope](https://www.rigolna.com/ds1z/)
* [Brüel & Kjær Type 3050-B-6](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/LAN-XI-data-acquisition-hardware/modules/type-3050)
* [DT9857E-16](https://www.mccdaq.com/Products/Sound-Vibration-DAQ/DT9857E)

The first step in remotely controlling these devices is to connect them to a computer via either USB or Ethernet.

### Front Panels

The three devices are connected to each other using [BNC](https://en.wikipedia.org/wiki/BNC_connector) connectors and cables.

You will require 4 BNC cables and 2 BNC Tee Adapters.

![Device configuration (front)](figures/Device_configuration_front.png)

### Rear panels (Ethernet)

The three devices can be connected to the network using Ethernet cables and an unmanaged switch.

You will require 4 Ethernet cables and an unmanaged switch.

![Device configuration (Ethernet)](figures/Device_configuration_ethernet.png)

The devices can either be connected directly to the computer via the switch, or the switch can be connected to a router with WiFi access for the computer.

### Rear panels (USB)

The signal generator and the oscilloscope can be connected to the computer via USB.
The DAQ will still require Ethernet connection.

You will require 1 Ethernet cable and 2 USB cables.

![Device configuration (USB)](figures/Device_configuration_usb.png)


## Connecting VISA compatible devices via USB

Connecting a device via USB does not require any configuration of the device, but will generally require configuration on the computer to which you are attaching the device. 
This configuration is different for different operating systems. Here we run through the configuration for Linux and Windows 10.

### Linux

Start by connecting the device to your computer via USB cable, and turning on the device.
Then get a list of currently connected USB devices:

In [ ]:
usb_devices = subprocess.check_output('lsusb').decode('UTF-8')
print(usb_devices)

Hopefully the device is contained in the list. In my case, the device is the 4th on the list, and from this I can obtain the Vendor ID and the Product ID:

In [ ]:
#Keysight 3500B Series
idVendor = '0957'
idProduct = '2807'

We now need to add the device to the list of devices that we can utilise (NB: this requires root access).

The line that we need to add is:

In [ ]:
import getpass
new_usb_rule = (f'SUBSYSTEMS=="usb", ACTION=="add", ' +
                f'ATTRS{{idVendor}}=="{idVendor}", ' +
                f'ATTRS{{idProduct}}=="{idProduct}", ' +
                f'GROUP="{getpass.getuser()}", MODE="0660"'
               )
print(new_usb_rule)

Check that the rule is not already there:

In [ ]:
!cat /etc/udev/rules.d/usbtmc.rules

If it is not already there, then add it by running the following commands in a terminal window:

In [ ]:
print(f"echo '{new_usb_rule}' | sudo tee -a /etc/udev/rules.d/usbtmc.rules")
print('sudo udevadm control --reload-rules && udevadm trigger')

In [ ]:
!pip install libusb1

In [ ]:
import usb1
with usb1.USBContext() as context:
    handle = context.openByVendorIDAndProductID(
        idVendor,
        idProduct,
        skip_on_error=True,
    )
    if handle is None:
        # Device not present, or user is not allowed to access device.
        print("Device not present")
    with handle.claimInterface(INTERFACE):
        # Do stuff with endpoints on claimed interface.
        #print(Agilent.ask("*IDN?"))
        pass

In [ ]:
import usbtmc
import usb.core
import usb.backend.libusb1
Agilent = usbtmc.Instrument(idVendor,idProduct)
print(Agilent.ask("*IDN?"))

In [ ]:
usbtmc?

In [ ]:
#Rigol DS1074Z
idVendor = 
idProduct = 

### Windows 10

In [ ]:
!pip install pyvisa

Install [NI VISA](https://pyvisa.readthedocs.io/en/latest/faq/getting_nivisa.html#faq-getting-nivisa)

In [ ]:
!lsusb

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()

print(rm.list_resources())

Agilent = rm.open_resource(rm.list_resources()[0])
print(Agilent.query("*IDN?"))

## Connecting vxi11 compatible devices via Ethernet

Connecting devices via Ethernet is generally easier than connecting via USB. There is no need to load any drivers or run any root or admin commands, however the computer that you are using and the devices themselves all need to be on the same WAN.



In [ ]:
!pip install python-vxi11

In [ ]:
keysight_ip = '192.168.0.55'

In [ ]:
import vxi11
import time
Keysight = vxi11.Instrument(keysight_ip)
print(Keysight.ask("*IDN?"))

In [ ]:
rigol_ip = '192.168.0.60'

In [ ]:
Rigol = vxi11.Instrument(rigol_ip)
print(Rigol.ask("*IDN?"))

## Interacting with a Brüel & Kjær Type 3050-B-6 using Python

To communicate with a stand-alone [Brüel & Kjær Type 3050-B-6](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/LAN-XI-data-acquisition-hardware/modules/type-3050), it is necessary to have a [Notar™ BZ-7848-A (LAN-XI stand-alone recorder license)](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/general-purpose-analyzer-system/lan-xi-notar), which allows you to interact with the device via a browser, utilising the Ethernet port at the back of the device. 

The details of how to interact with the BnK have been worked out in the [PyBnK](https://github.com/uwasystemhealth/PyBnK) respository, here we simply need to import the module, then use the BnK device.

In [ ]:
!cd ../PyBnK;pip install .;cd -

In [ ]:
from bnk.bnk import WavHeader, OpenWav, Instrument

In [ ]:
bnk_ip = '192.168.0.29'
ADAC = Instrument(bnk_ip)
print(ADAC) # Show some info about the BnK device

## Initialising the devices

### Signal Generator

The manual for the Keysight signal generator can be found here: https://literature.cdn.keysight.com/litweb/pdf/33500-90901.pdf

A local copy of the document can be found here: [references/33500-90901.pdf](references/33500-90901.pdf)

This document contains the SCPI Programming Reference section, which explains how to manipulate 
the device using ASCII commands.

SCPI (Standard Commands for Programmable Instruments) is an ASCII-based instrument command language 
designed for test and measurement instruments. 

We begin by outputting two sine waves.
We can specify the amplitude, frequency, offset and phase of these waves.

But first, we need to explain the impedance setting.

See https://www.keysight.com/main/editorial.jspx%3Fckey%3D1948055%26id%3D1948055%26nid%3D-11143.0.00%26lc%3Djpn%26cc%3DJP for details.

The impedance that you set on the signal generator depends upon what you intend to attach to the signal
generator.
If you intend to attach a high impedance device (such as an oscilloscope), then you would want
to set the signal generator to display the high impedance voltage level. 

WARNING: If you are powering the BnK using mains power, you are likely to pick up a lot of 50 Hz
noise on the oscilloscope while the BnK is in standby mode, but this problem disappears if the BnK
is placed into recorder mode (sometimes), or if powered by battery.

#### Keysight

In [ ]:
#enable 50 ohm impedance
#Keysight.write("OUTPut:LOAD 50") 
#Keysight.write("OUTPut2:LOAD 50")

#enable high impedance
Keysight.write("OUTPut:LOAD INF")
Keysight.write("OUTPut2:LOAD INF")

#Turn off channel 1
Keysight.write("OUTPut OFF")

#Turn off channel 2
Keysight.write("OUTPut2 OFF")

#Set the units to Vpp
Keysight.write("VOLTage:UNIT VPP")

In [ ]:
Keysight.write("TRIGger:SOURce IMMediate")
Keysight.write("BURSt:MODE TRIGgered")
Keysight.write("BURSt:INTernal:PERiod 1")

In [ ]:
import time

freq = [1000, 1000]
amp = [3,2]
offset = [0,0]
phase = [0,0]

#Turn off channel 1
Keysight.write("OUTPut OFF")
#Turn off channel 2
Keysight.write("OUTPut2 OFF")

time.sleep(1)

Keysight.write(f"APPLy:SIN {freq[0]},{amp[0]},{offset[0]}")
Keysight.write(f"SOURce2:APPLy:SIN {freq[1]},{amp[1]},{offset[1]}")
Keysight.write("PHAS:SYNC") #Sync the two internal channels
Keysight.write(f"PHASe {phase[0]}")
Keysight.write(f"SOURce2:PHASe {phase[1]}")


These commands are unnecessary here, as APPLy turns on output

In [ ]:
#Turn off channel 1
Keysight.write("OUTPut OFF")

In [ ]:
#Turn off channel 2
Keysight.write("OUTPut2 OFF")

In [ ]:
#Turn on channel 1
Keysight.write("OUTPut ON")

In [ ]:
#Turn on channel 2
Keysight.write("OUTPut2 ON")

#### Agilent

In [ ]:
#enable 50 ohm impedance
#Agilent.write("OUTPut:LOAD 50") 

#enable high impedance
Agilent.write("OUTPut:LOAD INF")

#Turn off output
Agilent.write("OUTPut OFF")

#Set the units to Vpp
Agilent.write("VOLTage:UNIT VPP")

In [ ]:
Agilent.write("TRIGger:SOURce IMMediate")
Agilent.write("BURSt:MODE TRIGgered")
Agilent.write("BURSt:INTernal:PERiod 1")

In [ ]:
import time

freq = [1000, 1000]
amp = [3,2]
offset = [0,0]
phase = [0,0]

#Turn off channel 1
Agilent.write("OUTPut OFF")

time.sleep(1)

Agilent.write(f"APPLy:SIN {freq[0]},{amp[0]},{offset[0]}")

These commands are unnecessary here, as APPLy turns on output

In [ ]:
#Turn off channel 1
Agilent.write("OUTPut OFF")

In [ ]:
#Turn on channel 1
Agilent.write("OUTPut ON")

### Oscilloscope

The manual for the Rigol can be found here: https://www.batronix.com/pdf/Rigol/UserGuide/DS1000Z_UserGuide_EN.pdf

A local copy of the document can be found here: [references/DS1000Z_UserGuide_EN.pdf](references/DS1000Z_UserGuide_EN.pdf)

The manual for the remotely programming the oscilloscope can be found here: http://www.fisica.unipg.it/~michele.pauluzzi/Laboratorio%20II%20varie/Rigol%20Oscilloscope%20DS1054Z%20Programming%20Guide.pdf

A local copy of the document can be found here: [references/Rigol%20Oscilloscope%20DS1054Z%20Programming%20Guide.pdf](references/Rigol%20Oscilloscope%20DS1054Z%20Programming%20Guide.pdf)

This document contains the SCPI Programming Reference section, which explains how to manipulate 
the device using ASCII commands.

In [ ]:
#Clear any messages which might be on the screen
Rigol.write(":DISPlay:CLEar")

In [ ]:
#Run Button
Rigol.write(":RUN")

In [ ]:
#Stop Button
Rigol.write(":STOP")

In [ ]:
#To turn on/off a channel from the display, use this syntax:
# ":CHANnel():DISPlay ()", First ()-Channel Number 1-4, Second () - ON/OFF 
Rigol.write(":CHANnel4:DISPlay OFF")

In [ ]:
#Turn on Channel 1
Rigol.write(":CHANnel1:DISPlay On")

In [ ]:
#Turn on Channel 2
Rigol.write(":CHANnel2:DISPlay On")

In [ ]:
#Turn on Channel 3
Rigol.write(":CHANnel3:DISPlay On")

In [ ]:
#Turn on Channel 4
Rigol.write(":CHANnel4:DISPlay On")

In [ ]:
#Turn off Channel 1
Rigol.write(":CHANnel1:DISPlay Off")

In [ ]:
#Turn off Channel 2
Rigol.write(":CHANnel2:DISPlay Off")

In [ ]:
#Turn off Channel 3
Rigol.write(":CHANnel3:DISPlay Off")

In [ ]:
#Turn off Channel 4
Rigol.write(":CHANnel4:DISPlay Off")

In [ ]:
from datetime import datetime

#Set the clock
Rigol.write(f"Date '{datetime.now().strftime('%Y-%m-%d')}'")
Rigol.write(f"Time '{datetime.now().strftime('%H:%M:%S')}'")

# Turn on Channel 1 and 2, turn off channel 3 and 4

# Configure the channels
for n in range(1,5):
    Rigol.write(f"CH{n}:Termination MEG") #Set termination to 1 MΩ 
    #Tek.write(f"CH{n}:Termination FIFTy") #Set termination to 50 Ω 
    Rigol.write(f"CH{n}:COUPling DC") #Set coupling to DC
    Rigol.write(f"CH{n}:INVErt OFF") #Set Invert to off
    Rigol.write(f"CH{n}:POSition 0") #Set vertical position to 0    

#Tek.write("CLEARMenu")

# Configure the vertical axis
for n in range(1,3):
    Rigol.write(f'CH{n}:YUN "V"') 
    Rigol.write(f"CH{n}:SCALE 500E-3")
    
# Configure the horizontal axis
Rigol.write("HORIZONTAL:SCALE 400E-6")

# Configure the trigger
Rigol.write("TRIGger:A:TYPE Edge")
Rigol.write("TRIGger:A:EDGE:SOUrce CH1")
Rigol.write("TRIGger:A:EDGE:COUPling DC")
Rigol.write("TRIGger:A:EDGE:SLOpe RISE")
Rigol.write("TRIGger:A:LEVel:CH1 0.1")
Rigol.write("TRIGger:A:MODe NORMal")

# Set some measurements
Rigol.write("MEASUrement:MEAS1:TYPe FREQuency")
Rigol.write("MEASUrement:MEAS2:TYPe FREQuency")
Rigol.write("MEASUrement:MEAS3:TYPe PK2Pk")
Rigol.write("MEASUrement:MEAS4:TYPe PK2Pk")

Rigol.write("MEASUrement:MEAS1:SOUrce CH1")
Rigol.write("MEASUrement:MEAS2:SOUrce CH2")
Rigol.write("MEASUrement:MEAS3:SOUrce CH1")
Rigol.write("MEASUrement:MEAS4:SOUrce CH2")

for n in range(1,5):
    Rigol.write(f"MEASUrement:MEAS{n}:SOUrce2 CH1")
    Rigol.write(f"MEASUrement:MEAS{n}:STATE ON")

In [ ]:
Rigol.write("MEASUrement:CLEARSNapshot")

In [ ]:
Rigol.ask("MEASUrement:MEAS1?")

### Capture a screenshot

The code below shows how you can capture a screenshot, such as this:

![Tektronix screenshot](figures/tektronix_screenshot.png)

In [ ]:
from pathlib import Path
from IPython.display import Image

Path("untracked").mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = f'untracked/tektronix_screenshot_{timestamp}.png' 

Rigol.write("SAVE:IMAG:FILEF PNG")
Rigol.write("HARDCOPY:INKSaver ON") #This does not work
Rigol.write("HARDCOPY START")
raw_data = Tek.read_raw()

with open(filename, 'wb') as file:
    file.write(raw_data)
print("Captured screenshot.")

Image(filename) 

#### Capture the raw data

See https://www.tek.com/support/faqs/programing-how-get-and-plot-waveform-dpo-mso-mdo4000-series-scope-python

In [ ]:
import numpy as np
from struct import unpack

def get_channel(Dev,channel):
    
    Dev.write(f"DATA:SOU CH{channel}")
    Dev.write("DATA:WIDTH 1")
    Dev.write("DATA:ENC RPB")

    ymult = float(Dev.ask("WFMPRE:YMULT?"))
    yzero = float(Dev.ask("WFMPRE:YZERO?"))
    yoff = float(Dev.ask("WFMPRE:YOFF?"))
    xincr = float(Dev.ask("WFMPRE:XINCR?"))

    Dev.write("CURVE?")
    data = Dev.read_raw()

    headerlen = 2 + int(data[1])
    header = data[:headerlen]
    ADC_wav = data[headerlen:-1]
    ADC_wav = np.array(unpack('%sB' % len(ADC_wav),ADC_wav))

    Volts = (ADC_wav - yoff) * ymult + yzero
    Time = np.arange(0, xincr * len(Volts), xincr)
    # The oscilloscope appears to capture 9952 points irrespective
    # of the time resolution, and to match the sample with t=0 on the
    # screen we need to shift the time back by 4952 points
    # (ie the data are not symmetrical about the middle)
    Time = Time - xincr*(len(Volts) - 5000)
    
    return Time, Volts


In [ ]:
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# create a new plot with a title and axis labels
p = figure(title="simple line example", 
           height=400,
           width=800,
           y_axis_label='Volts', 
           x_axis_label='Time',
          )

Time, Volts = get_channel(Tek,1)
# add a line renderer with legend and line thickness
p.line(Time, Volts, legend_label="1", line_width=2, line_color="Yellow")

Time, Volts = get_channel(Tek,2)
# add a line renderer with legend and line thickness
p.line(Time, Volts, legend_label="2", line_width=2, line_color="Cyan")

# show the results
show(p)

## Interacting with a Brüel & Kjær Type 3050-B-6 using Python

To communicate with a stand-alone [Brüel & Kjær Type 3050-B-6](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/LAN-XI-data-acquisition-hardware/modules/type-3050), it is necessary to have a [Notar™ BZ-7848-A (LAN-XI stand-alone recorder license)](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/general-purpose-analyzer-system/lan-xi-notar), which allows you to interact with the device via a browser, utilising the Ethernet port at the back of the device. 

The details of how to interact with the BnK have been worked out in the [PyBnK](https://github.com/uwasystemhealth/PyBnK) respository, here we simply need to import the module, then use the BnK device.

In [ ]:
bnk_ip = '169.254.245.20'

In [ ]:
from bnk.bnk import WavHeader, OpenWav, Instrument

ADAC = Instrument(bnk_ip)

In [ ]:
print(ADAC) # Show some info about the BnK device

In [ ]:
ADAC.disable_all()
ADAC.set_samplerate(131072)
ADAC.set_name('MTF_test')
ADAC.set_channel(channel=2, name='Reson', 
                 c_filter='7.0 Hz', c_range='10 Vpeak')
ADAC.set_channel(channel=4, name='Source', 
                 c_filter='7.0 Hz', c_range='10 Vpeak')
print(ADAC)

In [ ]:
ADAC.list_recordings()

In [ ]:
ADAC.powerup()
recording_id = ADAC.record(1)
ADAC.powerdown()
WAV_file = ADAC.get_wav(directory='untracked',recording_id=recording_id)
ADAC.delete_recording(recording_id=recording_id)
wav_data, metadata, json_data = OpenWav(WAV_file)#, verbose=True)

In [ ]:
from bokeh.plotting import figure, output_notebook, show
import numpy as np

output_notebook()

# create a new plot with a title and axis labels
p = figure(title="BnK recording", x_axis_label='Time', y_axis_label='Voltage')

xincr = 1/metadata['SampleRate']

Volts = wav_data[:,0]
Time = np.arange(0, xincr * len(Volts), xincr)

# add a line renderer with legend and line thickness
p.line(Time, Volts, legend_label="1", line_width=2, line_color="Yellow")

Volts = wav_data[:,1]

# add a line renderer with legend and line thickness
p.line(Time, Volts, legend_label="2", line_width=2, line_color="Cyan")

# show the results
show(p)

## TODO

* Load the PyBnK library


### Burst pulses

In [ ]:
#enable high impedance
Agilent.write("OUTPut:LOAD INF")
 
#Set the Frequency to 2000 Hz
frequency = 2000
Agilent.write("FUNCtion SINusoid")
Agilent.write("FREQuency {}".format(frequency))
 
#Set the amplitude to 2.273 Vrms
amplitude = 2.273
Agilent.write("VOLTage:UNIT VRMS")
Agilent.write("VOLTage {}".format(amplitude))
 
#Set the offset to 0
Agilent.write("VOLTage:OFFSet {}".format(0))

In [ ]:
 
ncycles = 10
int_period = 1
  
Agilent.write("BURSt:NCYCles {}".format(ncycles))
Agilent.write("BURSt:INTernal:PERiod {}".format(int_period))          
Agilent.write("BURSt:STATe ON")
  
for f in range(2000,11000,1000):
    print(f'Pulsing at {f}')
    Agilent.write("FREQuency {}".format(f))
    Agilent.write("OUTPut ON")
    time.sleep(12)
    Agilent.write("OUTPut OFF")
    time.sleep(8)

print("Finished")

### Bandwidth-limited white noise

In [ ]:
#enable high impedance
Agilent.write("OUTPut:LOAD INF")
Agilent.write("OUTPut OFF")
Agilent.write("FUNCtion NOISe")
Agilent.write("FUNCtion:NOISe:BANDWidth 1.0E+04")

Agilent.write("OUTPut ON")

## Testing with the BnK running off mains power

Below we show the effect of running the BnK off mains power, without opening the recorder application, 
which gives 50 Hz noise:

![50 Hz noise from BnK power supply](figures/tektronix_screenshot_20200625145743.png)

This noise problem seems to disappear if you simply start the BnK recorder application, and
does not reappear, even if the recorder application is closed, until the device is rebooted.

## Complicated example

Now let's do a complicated example where we start the BnK recording and then generate a variety of signals using
the signal generator, as well as gather a variety of data from the oscilloscope.

For this example, I am going to turn all filters off on the BnK.

In [ ]:
# Get the BnK ready

bnk_ip = '169.254.245.20'

from bnk.bnk import WavHeader, OpenWav, Instrument

ADAC = Instrument(bnk_ip)

ADAC.disable_all()
ADAC.set_samplerate(131072)
ADAC.set_name('Bench_test')
ADAC.set_channel(channel=2, name='SigGen1', 
                 c_filter='DC', c_range='10 Vpeak')
ADAC.set_channel(channel=4, name='SigGen2', 
                 c_filter='DC', c_range='10 Vpeak')

In [ ]:
# Get the SigGen ready

keysight_ip = '169.254.245.21'

import vxi11
import time

Agilent = vxi11.Instrument(keysight_ip)
print(Agilent.ask("*IDN?"))

#enable high impedance
Agilent.write("OUTPut:LOAD INF")
#Agilent.write("OUTPut2:LOAD INF")

#Turn off channel 1
Agilent.write("OUTPut OFF")

#Turn off channel 2
#Agilent.write("OUTPut2 OFF")

#Set the units to Vpp
Agilent.write("VOLTage:UNIT VPP")



In [ ]:
# Get the Oscilloscope ready

tektronix_ip = '169.254.4.115'

Tek = vxi11.Instrument(tektronix_ip)
print(Tek.ask("*IDN?"))

from datetime import datetime

#Set the clock
Tek.write(f"Date '{datetime.now().strftime('%Y-%m-%d')}'")
Tek.write(f"Time '{datetime.now().strftime('%H:%M:%S')}'")

# Turn on Channel 1 and 2, turn off channel 3 and 4
Tek.write("SELECT:CH1 ON")
Tek.write("SELECT:CH2 ON")
Tek.write("SELECT:CH3 OFF")
Tek.write("SELECT:CH4 OFF")

# Configure the channels
for n in range(1,5):
    Tek.write(f"CH{n}:Termination MEG") #Set termination to 1 MΩ 
    #Tek.write(f"CH{n}:Termination FIFTy") #Set termination to 50 Ω 
    Tek.write(f"CH{n}:COUPling DC") #Set coupling to DC
    Tek.write(f"CH{n}:INVErt OFF") #Set Invert to off
    Tek.write(f"CH{n}:POSition 0") #Set vertical position to 0    

#Tek.write("CLEARMenu")

# Configure the vertical axis
for n in range(1,3):
    Tek.write(f'CH{n}:YUN "V"') 
    Tek.write(f"CH{n}:SCALE 1")
    
# Configure the horizontal axis
Tek.write("HORIZONTAL:SCALE 400E-6")

# Configure the trigger
trig_level = 0.5
Tek.write("TRIGger:A:TYPE Edge")
Tek.write("TRIGger:A:EDGE:SOUrce CH1")
Tek.write("TRIGger:A:EDGE:COUPling DC")
Tek.write("TRIGger:A:EDGE:SLOpe RISE")
Tek.write(f"TRIGger:A:LEVel:CH1 {trig_level}")
Tek.write("TRIGger:A:MODe NORMal")

set_measurements = False
if set_measurements:
    # Set some measurements
    Tek.write("MEASUrement:MEAS1:TYPe FREQuency")
    Tek.write("MEASUrement:MEAS2:TYPe FREQuency")
    Tek.write("MEASUrement:MEAS3:TYPe PK2Pk")
    Tek.write("MEASUrement:MEAS4:TYPe PK2Pk")

    Tek.write("MEASUrement:MEAS1:SOUrce CH1")
    Tek.write("MEASUrement:MEAS2:SOUrce CH2")
    Tek.write("MEASUrement:MEAS3:SOUrce CH1")
    Tek.write("MEASUrement:MEAS4:SOUrce CH2")

    for n in range(1,5):
        Tek.write(f"MEASUrement:MEAS{n}:SOUrce2 CH1")
        Tek.write(f"MEASUrement:MEAS{n}:STATE ON")
else:
    #Remove all measurements
    for n in range(1,5):
        Tek.write(f"MEASUrement:MEAS{n}:STATE OFF")
    

In [ ]:
import numpy as np
from struct import unpack

def get_channel(Dev,channel):
    
    Dev.write(f"DATA:SOU CH{channel}")
    Dev.write("DATA:WIDTH 1")
    Dev.write("DATA:ENC RPB")

    ymult = float(Dev.ask("WFMPRE:YMULT?"))
    yzero = float(Dev.ask("WFMPRE:YZERO?"))
    yoff = float(Dev.ask("WFMPRE:YOFF?"))
    xincr = float(Dev.ask("WFMPRE:XINCR?"))

    Dev.write("CURVE?")
    data = Dev.read_raw()

    headerlen = 2 + int(data[1])
    header = data[:headerlen]
    ADC_wav = data[headerlen:-1]
    ADC_wav = np.array(unpack('%sB' % len(ADC_wav),ADC_wav))

    Volts = (ADC_wav - yoff) * ymult + yzero
    Time = np.arange(0, xincr * len(Volts), xincr)
    # The oscilloscope appears to capture 9952 points irrespective
    # of the time resolution, and to match the sample with t=0 on the
    # screen we need to shift the time back by 4952 points
    # (ie the data are not symmetrical about the middle)
    Time = Time - xincr*(len(Volts) - 5000)
    
    return Time, Volts


In [ ]:
Tek.write("CLEARMenu")

In [ ]:
Tek.write("FPANEL:PRESS SINGleseq")

In [ ]:
Tek.ask("TRIGger:STATe?")

In [ ]:
from pathlib import Path
from IPython.display import Image
from IPython.display import display
from math import log10, ceil, floor

Path("untracked").mkdir(exist_ok=True)

ADAC.powerup()
recording_id = ADAC.start_record()

start_frequency = 2000
stop_frequency = 10000
step_frequency = 1000
ncycles = 10
int_period = 1
nbursts = 3
pause_period = int_period*2

files = []

Agilent.write("FUNCtion SINusoid")
Agilent.write("FREQuency {}".format(start_frequency))

#Set the amplitude to 2.273 Vrms
amplitude = 2.273
Agilent.write("VOLTage:UNIT VRMS")
Agilent.write("VOLTage {}".format(amplitude))

Agilent.write("TRIGger:SOURce IMMediate")
Agilent.write("BURSt:MODE TRIGgered")
Agilent.write("BURSt:NCYCles {}".format(ncycles))
Agilent.write("BURSt:INTernal:PERiod {}".format(int_period))          
Agilent.write("BURSt:STATe ON")

oscilloscope_output = {}

# The horizontal scale on the tektronix is some exponent multiplied
# by 1, 2 or 4.
# To fit all the cycles on the screen after the trigger, we require
# ncycles/frequency < 5 * horizontal_scale
# Therefore
#   horizontal_scale > n_cycles/(5*frequency)
def ceil_124(x):
    ''' The Tektronix oscilloscope has a horizontal scale which increments
    from 1 to 2 to four and then to 10. This function returns the increment
    value greater than the given value.'''
    
    exp = floor(log10(abs(x)))
    coef = x/10**exp
    if coef < 1: #Should not be possible
        return 10**exp
    elif coef < 2:
        return 2*10**exp
    elif coef < 4:
        return 4*10**exp
    else:
        return 10**(exp+1)

t1 = datetime.now()

for f in range(start_frequency,stop_frequency+step_frequency,step_frequency):
    print(f'Pulsing at {f}')
    hoz_scale = ceil_124(ncycles/(5*f)) # at least 2.5 periods per division
    Tek.write(f"HORIZONTAL:SCALE {hoz_scale}") 
    Agilent.write("FREQuency {}".format(f))
    oscilloscope_output[f] = []
    Tek.write("FPANEL:PRESS SINGleseq")    
    
    t2 = datetime.now()
    run_time = (t2 - t1).total_seconds()
    if f == start_frequency:
        sleep_time = 0.2
    else:
        sleep_time = int_period*nbursts + pause_period - run_time
    time.sleep(sleep_time)

    Agilent.write("OUTPut ON")
    t1 = datetime.now()
    print(f"Last runtime was {run_time} seconds, slept for {sleep_time} seconds.")
    b = 0
    while b < nbursts:

        #It takes about 0.2 seconds to gather a pulse from the oscilloscope
        #so int_period needs to be longer than that
        while Tek.ask("TRIGger:STATe?") != 'SAVE':
            pass
        Time, Volts = get_channel(Tek,1)
        #print(f"Got snapshot {b}") 
        Tek.write("FPANEL:PRESS SINGleseq")
        oscilloscope_output[f].append([Time, Volts])
        while Tek.ask("TRIGger:STATe?") != 'ARMED':
            pass
        b = b + 1
    
    Agilent.write("OUTPut OFF")

print("Finished")

time.sleep(1)

ADAC.stop_record()

ADAC.powerdown()
WAV_file = ADAC.get_wav(directory='untracked',recording_id=recording_id)
ADAC.delete_recording(recording_id=recording_id)
wav_data, metadata, json_data = OpenWav(WAV_file)#, verbose=True)


In [ ]:
#import ipyplot
#ipyplot.plot_images(files, max_images=20, img_width=600)


In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import mpl

output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Example pulses", 
           height=800,
           width=1200,
           y_axis_label='Volts', 
           x_axis_label='Time',
          )

color = mpl['Plasma'][len(oscilloscope_output)]

xincr = 1/metadata['SampleRate']

Volts = wav_data[:,0]
Time = np.arange(0, xincr * len(Volts), xincr)

#This is the time from the beginning of the BnK recording to the
#beginning of the first burst in the first set
#Triggering off the BnK is difficult, because the sample rate is
#relatively low
f = start_frequency
amp = amplitude*np.sqrt(2)
trig_offset = np.arcsin(trig_level/amp)/(2*np.pi*f)
offset = Time[np.argmax(Volts>trig_level)] - trig_offset
print(f"Time to first pulse : {offset} seconds.")

plt_x_separation = 0.02
plt_y_separation = 7
plt_len = 1000
neg_shift = 40

#Agilent ECD-P-MD-32314
time_scale = 1.000052

#Keysight ECD-P-MD-32352
time_scale = 1.

# The Agilent Sig Gen is working well, but the Keysight is not.
# All pulses (even between bursts) are separated by integer multiples
# of the int_period for the Agilent. Not so for the Keysight.
# I don't know if there is a setting which needs changing or this is 
# just a limitation of the device.

n = 0 
for f, d_list in oscilloscope_output.items():
    trig_offset = np.arcsin(trig_level/amp)/(2*np.pi*f)
    for x, d in enumerate(d_list):
        p.line(d[0] + trig_offset + x*plt_x_separation,
               d[1] + plt_y_separation*n,
               legend_label=f"{f} Hz",
               line_color=color[n]
              )
        start = n*(int_period*nbursts + pause_period)*time_scale #this covers the breaks
        start = start + x*int_period*time_scale + offset
        start_pos = (np.abs(Time - start)).argmin()
        p.circle(Time[start_pos-neg_shift:start_pos+plt_len]-Time[start_pos] + x*plt_x_separation,
                 Volts[start_pos-neg_shift:start_pos+plt_len] + plt_y_separation*n, 
                 legend_label=f"{f} Hz BnK", 
                 line_width=0, 
                 fill_color=color[n],
                )
    n = n + 1

# add a line renderer with legend and line thickness
#p.line(Time, Volts, legend_label="BnK Channel 1", line_width=2, line_color="Black")

p.legend.click_policy="hide"

#p.x_range.start = -0.2e-3
#p.x_range.end = 1.2e-3
#p.y_range.start = -5
#p.y_range.end = 60

# show the results
show(p)